# Intro

Project developed as portfolio.

The objective of this project is to scrap the site https://www.gbvg.uk/ in order to obtain all Grat British vineyards, and export to css.

## Website scrapping

In [71]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd
from bs4 import BeautifulSoup
import time
import seaborn as sns
import matplotlib as plt
%matplotlib inline
sns.set_style(style = 'whitegrid')

In [ ]:
cards = []
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','y','w']

for letter in alphabet:
    time.sleep(3)
    url = 'https://www.gbvg.uk/glossary/' + letter
    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    try:
        req = Request(url, headers = header)
        response = urlopen(req)
        html = response.read()

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
      print(e.reason)

    #html cleaning
    html = html.decode('utf-8')
    html = " ".join(html.split()).replace('> <', '><')
    soup = BeautifulSoup(html, 'html.parser')

    vineyards = soup('tr', {'class':'odd'}) + soup('tr', {'class':'even'})
    
    for vineyard in vineyards:
        card = {}

        card['vineyard'] = vineyard.find('h3').getText()
        card['vy_slug'] = vineyard.a.get('href')
        try:
            card['img_link'] = vineyard.img.get('src')
        except:
            pass

        try:
            card['locality'] = vineyard.find('span', {'class':'locality'}).getText()
        except:
            pass

        try:
            card['phone'] = vineyard.find('span', {'class':'value'}).getText()
        except:
            pass

        try:
            card['location'] = vineyard.find('div', {'class':'location-hidden'}).getText()
        except:
            pass

        try:
            card['postal_code'] = vineyard.find('td', {'class':'views-field-postal-code'}).getText()
        except:
            pass

        try:
            card['province'] = vineyard.find('td', {'class':'views-field-province'}).getText()
        except:
            pass

        try:
            card['hectares'] = vineyard.find('td', {'class':'views-field-field-hectares'}).getText()
        except:
            pass

        try:
            card['verified'] = vineyard.find('td', {'class':'views-field-field-verified'}).getText()
        except:
            pass

        cards.append(card)


In [58]:
vineyards = pd.DataFrame(cards)
vineyards.head()

,vineyard,vy_slug,locality,phone,location,postal_code,province,hectares,verified,img_link
0,Abberley Hills Vineyard,node/38,Worcester,01299 896412,Worcestershire,WR6 6JP,Worcestershire,0.2000 :,,NaN
1,Abbey Vineyards - Granby Vineyard,/vineyard/abbey-vineyards-granby-vineyard,Granby,NaN,Nottinghamshire,NG13 9PU,Nottinghamshire,1.2500 :,,https://www.gbvg.uk/sites/default/files/styles...
2,Abbey Vineyards - Lyndon Top Vineyard (Rutland...,/vineyard/abbey-vineyards-lyndon-top-vineyard-...,Oakham,01476 550 191,Leicestershire,LE15 8RN,Leicestershire,1.6194 :,,https://www.gbvg.uk/sites/default/files/styles...
3,Abbey Vineyards - Windy Ridge Vineyard (East B...,/vineyard/abbey-vineyards-windy-ridge-vineyard...,East Bridgford,NaN,Nottinghamshire,NG13 8LP,Nottinghamshire,3.0000 :,,https://www.gbvg.uk/sites/default/files/styles...
4,Absent Friends Allotment Vines,node/47,Stapleford,01223 842486,Cambridgeshire,CB22 5BT,Cambridgeshire,0.0170 :,,NaN


In [59]:
vineyards.describe()

,vineyard,vy_slug,locality,phone,location,postal_code,province,hectares,verified,img_link
count,691,691,670,516,685,691,691,691,691,500
unique,691,691,405,496,60,665,61,319,3,500
top,Abberley Hills Vineyard,node/38,Exeter,+44 (0) 1798 813989,Kent,,Kent,,,https://www.gbvg.uk/sites/default/files/styles...
freq,1,1,12,6,64,8,64,77,551,1


The top locality with **12** Vineyards is **Exceter**

THe top location and province, with 64 vineyards is **Kent**

In [74]:
vineyards.to_csv('vineyards.csv')